In [5]:
# Problem data
# ------------

# Radiative forcing
sw_down = 700. # W/m2
lw_down = 200. # W/m2
albedo = 50. # %

# Aerodynamic resistance
ra = 50. # s/m

# Surface resistance
rsrf = 200. # s/m
# relative soil humidity
alpha = 0.8

# Air temperature and relative humidity (%)
temp_air = 20. # degC
rh = 80. # %

# Soil temperature at depht depth_soil, and thermal conductivity
temp_soil = 30. # degC
depth_soil = 0.1 # m
k_soil = 1.5 # W/m/degC

# Air density, air heat capacity at constant pressure, and water latent heat of vaporization
rho = 1.25 # kg/m3
cp = 1005. # J/kg/K
le = 2.4e6 # J/kg

# Stefan-Boltzman's constant
sigma = 5.67e-8 # W/m2/K4

In [6]:
# Water vapour pressure of saturated air
# See https://www.engineeringtoolbox.com/water-vapor-saturation-pressure-air-d_689.html

from math import exp

def psat(temp_deg, pres):
    temp_kelvin = temp_deg + 273.15
    a, b, c, d = 77.345, 0.0057, 7235., 8.2
    pws = exp(a + b*temp_kelvin - c/temp_kelvin) / (temp_kelvin**d)
    dpwsdt = (b + c/temp_kelvin**2 - d/temp_kelvin) * pws
    return pws, dpwsdt

# Specific humidity of saturated air and its derivative
def qsat(temp_deg, pres):
    temp_kelvin = temp_deg + 273.15
    beta, gamma = 0.622, 0.378
    pws, dpwsdt = psat(temp_deg, pres)
    q = beta*pws/(pres-gamma*pws)
    dq = beta/(pres-gamma*pws)*(1. + gamma*pws/(pres-gamma*pws))*dpwsdt
    return q, dq

In [7]:
# Energy balance
# --------------

from math import log10

def ebalance_surface(sw_down, lw_down, ra, rsrf, alpha, temp_air, rh, temp_soil):
    
    # Initial values
    q_air =rh/100.*qsat(temp_air, 101325.)[0]
    temp_g = temp_air
    dtemp = 1e3
    
    niter = 0
    
    rad_forcing = sw_down * (1-albedo/100.) + lw_down
    
    # Begin Newton Rhapson iteration
    while abs(dtemp) > 0.02:
        
        q_g, dq_g = qsat(temp_g, 101325.)
        q_g, dq_g = alpha*q_g, alpha*dq_g
        
        lw_up = sigma*(temp_g + 273.15)**4
        sh_up = -rho*cp * (temp_air - temp_g)/ra
        lh_up = -rho*le * (q_air - q_g)/(ra+rsrf)
        gh = -k_soil/depth_soil * (temp_soil - temp_g)
        
        f = -rad_forcing + lw_up + lh_up + sh_up + gh
        df = 4.*sigma*(temp_g+273.15)**3 + rho*cp/ra + rho*le/(ra+rsrf)*dq_g + k_soil/depth_soil
        
        dtemp = - f/df
        temp_g1 = temp_g + dtemp
        temp_g = temp_g1
        niter += 1
    
    print("Number of iterations: " + str(niter))
    print("Precision (log10) = " + str(round(log10(abs(dtemp)),2)))
    print("Surface temperature = " + str(round(temp_g,2)) + " degC")
    print("----")
    print("Shortwave_up = " + str(round(sw_down*albedo/100.,2)) + " W/m2")
    print("Longwave up = " + str(round(lw_up,2)) + " W/m2")
    print("Sensible heat = " + str(round(sh_up,2)) + " W/m2")
    print("Latent heat = " + str(round(lh_up,2)) + " W/m2")
    print("Ground heat = " + str(round(gh,2)) + " W/m2")      

In [8]:
ebalance_surface(sw_down, lw_down, ra, rsrf, alpha, temp_air, rh, temp_soil)

Number of iterations: 3
Precision (log10) = -3.98
Surface temperature = 25.03 degC
----
Shortwave_up = 350.0 W/m2
Longwave up = 448.21 W/m2
Sensible heat = 126.31 W/m2
Latent heat = 50.08 W/m2
Ground heat = -74.59 W/m2
